In [1]:
%pylab inline

import matplotlib.pyplot as plt
import requests
import zipfile
import re
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe, vocab


from functools import partial,reduce
from tqdm import tqdm, trange
tqdm = partial(tqdm, position=0, leave=True)
trange = partial(trange, position=0, leave=True)


DEVICE = 'cuda:0'

Populating the interactive namespace from numpy and matplotlib


In [2]:
url = 'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip'

r = requests.get(url)

ul = url.split('/')
name = ul[len(ul) - 1]

with open(name, 'wb') as file:
  file.write(r.content)

with zipfile.ZipFile(name, "r") as zip_ref:
  zip_ref.extractall("./")

!mv 'cornell movie-dialogs corpus' 'data'
!ls 'data'

 chameleons.pdf			 movie_lines.txt
'cornell movie-dialogs corpus'	 movie_titles_metadata.txt
 movie_characters_metadata.txt	 raw_script_urls.txt
 movie_conversations.txt	 README.txt


In [4]:
FIELD_SPLITTER = '+++$+++'

MAX_SAMPLES = 50000
MAX_LENGTH = 40

UNK_TOKEN = '<unk>'
PAD_TOKEN = '<PAD>'
BOS_TOKEN = '<BOS>'
EOS_TOKEN = '<EOS>'

UNK_TOKEN_IND = 0
PAD_TOKEN_IND = 1
BOS_TOKEN_IND = 2
EOS_TOKEN_IND = 3

BATCH = 4

In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [7]:
text_transform = lambda x, voc, tokenizer: [voc['<BOS>']] + [voc[token] for token in tokenizer(x)] + [voc['<EOS>']]

In [8]:
# this is terrible as fuck because torchtext is terrible as fuck
def load_conversations(path_to_movie_lines, path_to_movie_conversations):
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.replace('\n', '').split(' +++$+++ ')
            id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(path_to_movie_conversations, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.replace('\n', '').split(' +++$+++ ')
            conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]
            for i in range(len(conversation) - 1):
                inputs.append(preprocess_sentence(id2line[conversation[i]]))
                outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))
                if len(inputs) >= MAX_SAMPLES:
                    return inputs, outputs
    return inputs, outputs


In [9]:
def get_dataloader(path_to_movie_lines,
                   path_to_movie_conversations):
    questions, answers = load_conversations(path_to_movie_lines, path_to_movie_conversations)

    tokenizer = get_tokenizer('basic_english')

    counter = Counter()
    for sent in questions + answers:
        counter.update(tokenizer(sent))

    voc = vocab(counter)
    voc.insert_token(token=UNK_TOKEN, index=UNK_TOKEN_IND)
    voc.set_default_index(index=UNK_TOKEN_IND)
    voc.insert_token(token=PAD_TOKEN, index=PAD_TOKEN_IND)
    voc.insert_token(token=BOS_TOKEN, index=BOS_TOKEN_IND)
    voc.insert_token(token=EOS_TOKEN, index=EOS_TOKEN_IND)

    q_tokenized = [text_transform(t, voc, tokenizer) for t in questions]
    a_tokenized = [text_transform(t, voc, tokenizer) for t in answers]

    import tensorflow as tf # todo
    q_padded = tf.keras.preprocessing.sequence.pad_sequences(
        q_tokenized, maxlen=MAX_LENGTH, padding='post', value=1.0)

    a_padded = tf.keras.preprocessing.sequence.pad_sequences(
        a_tokenized, maxlen=MAX_LENGTH, padding='post', value=1.0)

    print("Vocab len", len(voc))

    dataloader = DataLoader(list(zip(q_padded, a_padded)), batch_size=BATCH, shuffle=False)

    print(voc)
    torch.save(voc, 'vocab')

    return dataloader, text_transform, voc


In [12]:
lines_path = 'data/movie_lines.txt'
conversations_path = 'data/movie_conversations.txt'    

dataloader, text_transform, voc = get_dataloader(lines_path,
                                                 conversations_path)

Vocab len 23068
Vocab()


In [15]:
for i,x in enumerate(dataloader):
  if i > 1: break
  print(x[0].shape)

torch.Size([4, 40])
torch.Size([4, 40])


In [ ]:
!cat data/movie_conversations.txt | tail -n 10

In [ ]:
!ls data

In [ ]:
!cat data/README.txt

In [ ]:
# # trash




# def preprocess(x):
#   x_no_new = x.replace('\n', '')
#   text = x_no_new.split(FIELD_SPLITTER).pop()
#   embedding = g_vectors.get_vecs_by_tokens(tokenizer(text), lower_case_backup=True)
#   return embedding

# tokenizer = get_tokenizer('basic_english')
# g_vectors = GloVe(name='840B')
# g_vocab = vocab(g_vectors.stoi)


# train_iter = tt.data.BucketIterator(
#   dataset=train_obj,
#   batch_size = 2,
#   sort_key=lambda x: len(x.review),
#   shuffle=True,
#   device=DEVICE
# )

# trainloader = torch.utils.data.DataLoader(
# 	,
# 	batch_size=BATCH,
# 	num_workers=12,
# 	shuffle=True
# )

In [ ]:

# embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"),
#                                                lower_case_backup=True)
# embeddings
# 
# 
# 
# def batch(iterable, size):
#     from itertools import chain, islice
#     iterator = iter(iterable)
#     for first in iterator:
#         yield list(chain([first], islice(iterator, size - 1)))